#Part One - Scrape the list of US presidents from https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States. 
using pandas and save them as a CSV.

In [3]:
import pandas as pd

In [4]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [128]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States')

In [129]:
#tables[0]

In [7]:
tables[0].to_csv('presidents.csv', index = False)

PART TWO - Scrape the content of https://www.lemonde.fr/Links to an external site. and save it as a CSV.

We want: titles, subhead, article URL, whether it's premium or not, byline, article type, image URL.

You're going to be missing some article URLs - you should use try/except on them to avoid them being a problem, buuuut after that I would recommend printing the element to see why your approach isn't working (it's 100% possible to get all of the URLs!)
If you want to make sure the element you're getting has a specific attribute, you can do something like .select_one("a[href]"). That will only give you anchor tags (links) that have hrefs.
Instead of yes/no for the premium question, you can think of it as "put text in this column if the article is premium, don't put text in it if it is not"

In [54]:
from bs4 import BeautifulSoup
import requests

response = requests.get("https://www.lemonde.fr/en/")
doc = BeautifulSoup(response.text)

In [89]:
items = doc.find_all(class_='article')

In [127]:
rows = []
for item in items:
    row = {}
    row['title'] = item.find(class_='article__title').text
    try:
        row['url'] = item.find(class_="article article--nav").get('href')
    except:
        row['url'] = item.get('href', None)
    try:
        row['premium'] = item.find(class_='icon__premium').text
    except:
        pass
    try:
        row['image url'] = item.find(class_='lmd-link-clickarea__link').get('href')
    except:
        row['image url'] = item.get('href', None)
    try:
        row['subhed'] = item.find(class_="article__desc").text
    except:
        pass
    try:
        row['article__type'] = item.find(class_='article__type').text
    except: 
        pass
    try:
        row['article__byline'] = item.find(class_='article__type').text
    except: 
        pass
    rows.append(row)

In [126]:
import pandas as pd
df = pd.json_normalize(rows)
df.head()
df.to_csv("lemonde.csv", index=False)

PART 3 -
Scrape the list of third party drivers license locations from https://travel-id-documents.az.gov/authorized-third-party-driver-license-locationsLinks to an external site. but include the link. Save as a CSV. Since it's more than just the text from the table, this requires actually using BeautifulSoup :(

Tips:

You'll need user-agent headers for this one
In class we always did something like item.find('h2') or the like because there was only ever one h2 we wanted. In this case there are four <td> tags that you want the text from! You'll need to use .find_all and treat them like normal lists ("give me the text from the first one," + "give me the text from the second one" etc etc)
It's okay to have columns you don't want or need! You can always remove them later

In [130]:
response = requests.get("https://travel-id-documents.az.gov/authorized-third-party-driver-license-locations")
doc = BeautifulSoup(response.text)

In [169]:
table = doc.find(class_='table')
rows = table.select('tr')

In [215]:
data = []
for row in rows[1:]:
  result = []
  if len(row.select('a')) > 0:
    result.append(row.select('a')[0]['href'])
  else:
    result.append('')
  cells = row.select('td')
  for cell in cells:
      result.append(cell.text.strip())
  data.append(result)

In [233]:
df= pd.DataFrame(data)
df.columns = ['link', 'company', 'address', 'telephone', 'hours']
df.to_csv("driverslicenselocations.csv", index=False)

In [232]:
df.to_csv("driverslicenselocations.csv", index=False)